# Post Processing

In [ ]:
import gzip
import pickle
from yaml import Loader, Dumper
import yaml

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
from coffea import hist
from plots.helpers import makePlot2
from tools.helpers import cutflow_scale_and_merge, getCutFlowTable
from yahist import Hist1D, Hist2D

## Import Histograms

In [ ]:
path = '../analysis/histos/for_plotting.pkl.gz'
with gzip.open(path) as fin:
    scaled_output = pickle.load(fin)

## Setting Up Plots

In [ ]:
N_bins = hist.Bin('multiplicity', r'$N$', 6, -0.5, 5.5)
mass_bins = hist.Bin('mass', r'$M\ (GeV)$', 40, 0, 400)
ht_bins = hist.Bin('pt', r'$H_{T}\ (GeV)$', 60, 0, 3000)
pt_bins = hist.Bin('pt', r'$p_{T}\ (GeV)$', 80, 200, 1000)
eta_bins = hist.Bin("eta", "$\eta$", 33, -4, 4)
phi_bins = hist.Bin("phi", "$\phi$", 33, -4, 4)
deltaR_bins = hist.Bin("deltaR", "$\DeltaR$", 10, 0, 1)
tau1_bins = hist.Bin("tau", "$\tau_1$", 10, 0, 0.7)
tau2_bins = hist.Bin("tau", "$\tau_2$", 10, 0, 0.5)
tau3_bins = hist.Bin("tau", "$\tau_3$", 10, 0, 0.4)
tau4_bins = hist.Bin("tau", "$\tau_4$", 10, 0, 0.3)

labels ={
            ('QCD_bEnriched_HT',): r'$QCD\ b-enriched\ (binned\ by\ HT)$',
            ('ZJetsToNuNu_HT',): r'$ZJets\to\nu\nu\ (binned\ by\ HT)$',
            ('WJetsToLNu_Njet',): r'$WJets\to L\nu\ (binned\ by\ N_{jets})$',
            ('TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU',): r'$t\bar{t}$',
            ('2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500',): '2HDMa_1500_750_10',
            ('2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750',): '2HDMa_1750_750_10',
            ('2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000',): '2HDMa_2000_750_10',
        }

colors ={
            ('QCD_bEnriched_HT',): '#D23FFE',
            ('ZJetsToNuNu_HT',): '#6BFE3F',
            ('WJetsToLNu_Njet',): '#FED23F',
            ('TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU',): '#FE3F6B',
        }
        
signals = [
            ('2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500',),
            ('2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750',), 
            ('2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000',),
        ]

plot_dir = '/home/users/$USER/public_html/HbbMET/plots/'

In [ ]:
tmp1 = scaled_output['MT_vs_sdmass']
        
keys = tmp1.values().keys()

histos1 = {}

for sample in keys:
    h1 = Hist1D.from_bincounts(
        tmp1.sum('mt', overflow='over').values(overflow = 'over')[sample].T,
        (tmp1.axis('mass').edges(overflow = 'over')),
        #errors = np.sqrt(tmp1.sum('pt', 'dataset', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
        )
    histos1[sample] = h1

backgrounds = []
for sample in keys:
    if sample not in signals:
        backgrounds += (sample,)
           
fig, (ax) = plt.subplots(figsize=(10,10))
hep.cms.label(
    'Preliminary',
    loc=0,
    ax=ax,
    #lumi = 3000,
    rlabel = '14 TeV',
    )

hep.histplot(
    [histos1[sample].counts for sample in backgrounds],
    histos1[list(keys)[0]].edges,
    #w2=[(hists[x].errors)**2 for x in keys ],
    histtype="step",
    density = True,
    stack=False,
    label=[labels[sample] for sample in backgrounds],
    color=[colors[sample] for sample in backgrounds],
    ax=ax
    )

hep.histplot(
    [histos1[sample].counts for sample in signals],
    histos1[list(keys)[0]].edges,
    w2=[(histos1[sample].errors)**2 for sample in signals],
    histtype="step",
    density = True,
    stack=False,
    label=[labels[sample] for sample in signals],
    ax=ax
        )

ax.set_xlabel(r'$softdrop\ mass\ (GeV)$')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
ax.legend(prop={'size': 12})

#fig.savefig('/home/users/ewallace/public_html/HbbMET/plots/AK8_sdmass_BL.png')

In [ ]:
makePlot2(scaled_output, 'met_pt', 'pt', met_bins, r'$MET_{pt}\ (GeV)$', labels, colors, signals, plot_dir)
makePlot2(scaled_output, 'met_pt_BL', 'pt', met_bins, r'$MET_{pt}\ (GeV)$', labels, colors, signals, plot_dir)
makePlot2(scaled_output, 'dphi_AK4_MET', 'phi', phi_bins2, r'$\Delta\varphi$', labels, colors, signals, plot_dir)
makePlot2(scaled_output, 'dphi_AK4_MET_BL', 'phi', phi_bins2, r'$\Delta\varphi$', labels, colors, signals, plot_dir)
makePlot2(scaled_output, 'dphi_AK8_MET', 'phi', phi_bins2, r'$\Delta\varphi$', labels, colors, signals, plot_dir)
makePlot2(scaled_output, 'dphi_AK8_MET_BL', 'phi', phi_bins2, r'$\Delta\varphi$', labels, colors, signals, plot_dir)
makePlot2(scaled_output, 'AK4_QCD_veto', 'phi', phi_bins2, r'$\Delta\varphi$', labels, colors, signals, plot_dir)
makePlot2(scaled_output, 'AK4_QCD_veto_BL', 'phi', phi_bins2, r'$\Delta\varphi$', labels, colors, signals, plot_dir)
makePlot2(scaled_output, 'AK8_QCD_veto', 'phi', phi_bins2, r'$\Delta\varphi$', labels, colors, signals, plot_dir)
makePlot2(scaled_output, 'AK8_QCD_veto_BL', 'phi', phi_bins2, r'$\Delta\varphi$', labels, colors, signals, plot_dir)

## Cutflow

In [ ]:
path = '../analysis/histos/output_flat.pkl.gz'
with gzip.open(path) as fin:
    output_flat = pickle.load(fin)

In [ ]:
with open('../data/samples.yaml', 'r') as f:
    samples = yaml.load(f, Loader = Loader)

In [ ]:
fileset = {
    'TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU': samples['TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU']['skim'],
    'ZJetsToNuNu_HT-100To200_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-100To200_14TeV-madgraph_200PU']['skim'],
    'ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU']['skim'],
    'ZJetsToNuNu_HT-400To600_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-400To600_14TeV-madgraph_200PU']['skim'],
    'ZJetsToNuNu_HT-600To800_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-600To800_14TeV-madgraph_200PU']['skim'],
    'ZJetsToNuNu_HT-800To1200_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-800To1200_14TeV-madgraph_200PU']['skim'],
    'ZJetsToNuNu_HT-1200To2500_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-1200To2500_14TeV-madgraph_200PU']['skim'],
    #'W0JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['W0JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
    'W1JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['W1JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
    'W2JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['W2JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
    'W3JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['W3JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
    'QCD_bEnriched_HT1000to1500_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['QCD_bEnriched_HT1000to1500_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
    'QCD_bEnriched_HT1500to2000_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['QCD_bEnriched_HT1500to2000_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
    'QCD_bEnriched_HT2000toInf_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['QCD_bEnriched_HT2000toInf_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
    'QCD_bEnriched_HT200to300_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['QCD_bEnriched_HT200to300_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
    'QCD_bEnriched_HT300to500_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['QCD_bEnriched_HT300to500_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
    'QCD_bEnriched_HT500to700_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['QCD_bEnriched_HT500to700_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
    'QCD_bEnriched_HT700to1000_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['QCD_bEnriched_HT700to1000_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
    '2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['ntuples'],
    '2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750']['ntuples'],
    '2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000']['ntuples'],
    '2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500': samples['2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['ntuples'],
    '2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750': samples['2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_1750_MH4_750_MH2_1750_MHC_1750']['ntuples'],
    '2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000': samples['2HDMa_gg_sinp_0.35_tanb_1.0_mXd_10_MH3_2000_MH4_750_MH2_2000_MHC_2000']['ntuples'],
}

In [ ]:
meta = {}

for sample in fileset:
    meta[sample] = output_flat[sample]
    meta[sample]['xsec'] = samples[sample]['xsec']
    meta[sample]['nevents'] = samples[sample]['nevents']

In [ ]:
output_flat['cutflow']

In [ ]:
cutflow = cutflow_scale_and_merge(output_flat['cutflow'], meta, fileset, 3000)

In [ ]:
cutflow

In [ ]:
proceses = ['ZJetsToNuNu_HT', 'WJetsToLNu_Njet', 'QCD_bEnriched_HT', 'TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU', '2HDMa_1500_750', '2HDMa_1750_750', '2HDMa_2000_750']
cuts = ['total', 'lepton_veto', 'MET>300', 'N_AK8>0', 'N_AK4>1', 'min_AK8_pt', 'dphi_AK8_MET>1', '1<dphi_AK4_MET<3', 'AK4_QCD_veto', 'AK8_QCD_veto', 'on_H', 'MT>600',  'N_H>0',]

In [ ]:
getCutFlowTable(cutflow, proceses, cuts, significantFigures=4, absolute=True, signal = ['2HDMa_1500_750', '2HDMa_1750_750', '2HDMa_2000_750'])

In [ ]:
getCutFlowTable(cutflow, proceses, cuts, significantFigures=4, absolute=True, signal = ['2HDMa_1500_750', '2HDMa_1750_750', '2HDMa_2000_750'])

# Data Cards

## 2HDMa_1500_750

In [ ]:
MT_vs_sdmass_1500_750 = Hist2D.from_bincounts(
    scaled_output['MT_vs_sdmass'].values(overflow='over')[('2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500',)].T,
    (scaled_output['MT_vs_sdmass'].axis('mt').edges(overflow='over'), scaled_output['MT_vs_sdmass'].axis('mass').edges(overflow='over')),
    errors = np.sqrt(scaled_output['MT_vs_sdmass'].values(sumw2=True, overflow='over')[('2HDMa_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500',)][1].T),
)

In [ ]:
for b in range(0,4):
    for j in range(0,10):
        if MT_vs_sdmass_1500_750.counts[b][j] == 0:
            MT_vs_sdmass_1500_750.counts[b][j] = 0.1
            MT_vs_sdmass_1500_750.errors[b][j] = 0.1

In [ ]:
MT_vs_sdmass_tt = Hist2D.from_bincounts(
     scaled_output['MT_vs_sdmass'].values(overflow='over')[('TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU',)].T,
    (scaled_output['MT_vs_sdmass'].axis('mt').edges(overflow='over'), scaled_output['MT_vs_sdmass'].axis('mass').edges(overflow='over')),
    errors = np.sqrt(scaled_output['MT_vs_sdmass'].values(sumw2=True, overflow='over')[('TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU',)][1].T),
)

In [ ]:
for b in range(0,4):
    for j in range(0,10):
        if MT_vs_sdmass_tt.counts[b][j] == 0:
            MT_vs_sdmass_tt.counts[b][j] = 0.1
            MT_vs_sdmass_tt.errors[b][j] = 0.1

In [ ]:
MT_vs_sdmass_W = Hist2D.from_bincounts(
     scaled_output['MT_vs_sdmass'].values(overflow='over')[('WJetsToLNu_Njet',)].T,
    (scaled_output['MT_vs_sdmass'].axis('mt').edges(overflow='over'), scaled_output['MT_vs_sdmass'].axis('mass').edges(overflow='over')),
    errors = np.sqrt(scaled_output['MT_vs_sdmass'].values(sumw2=True, overflow='over')[('WJetsToLNu_Njet',)][1].T),
)

In [ ]:
for b in range(0,4):
    for j in range(0,10):
        if MT_vs_sdmass_W.counts[b][j] == 0:
            MT_vs_sdmass_W.counts[b][j] = 0.1
            MT_vs_sdmass_W.errors[b][j] = 0.1

In [ ]:
MT_vs_sdmass_QCD = Hist2D.from_bincounts(
     scaled_output['MT_vs_sdmass'].values(overflow='over')[('QCD_bEnriched_HT',)].T,
    (scaled_output['MT_vs_sdmass'].axis('mt').edges(overflow='over'), scaled_output['MT_vs_sdmass'].axis('mass').edges(overflow='over')),
    errors = np.sqrt(scaled_output['MT_vs_sdmass'].values(sumw2=True, overflow='over')[('QCD_bEnriched_HT',)][1].T),
)

In [ ]:
for b in range(0,4):
    for j in range(0,10):
        if MT_vs_sdmass_QCD.counts[b][j] == 0:
            MT_vs_sdmass_QCD.counts[b][j] = 0.1
            MT_vs_sdmass_QCD.errors[b][j] = 0.1

In [ ]:
from tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('lumi',        'lnN')

binnum = 0

for b in range(0,4):
    for j in range(0,10):
        binname = 'bin'+str(binnum)
        Binname = str(b+1)+'_'+str(j+1)
        binnum += 1
        c.addBin(binname, ['QCD', 'Wjets', 'ttjets'], Binname) # signal is automatically added
        
        processes = {'signal': MT_vs_sdmass_1500_750, 'QCD': MT_vs_sdmass_QCD, 'Wjets': MT_vs_sdmass_W, 'ttjets': MT_vs_sdmass_tt}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(MT_vs_sdmass_1500_750.counts[b][j], 3))
        c.specifyExpectation(binname, 'QCD',    round(MT_vs_sdmass_QCD.counts[b][j], 3))
        c.specifyExpectation(binname, 'Wjets',    round(MT_vs_sdmass_W.counts[b][j], 3))
        c.specifyExpectation(binname, 'ttjets',    round(MT_vs_sdmass_tt.counts[b][j], 3))

        c.specifyObservation(binname, round(MT_vs_sdmass_QCD.counts[b][j] + MT_vs_sdmass_W.counts[b][j] + MT_vs_sdmass_tt.counts[b][j], 3))
        
c.specifyFlatUncertainty('lumi', 1.01)
c.writeToFile('./2HDMa_1500_750_1.txt')

In [ ]:
res = c.calcLimit('./2HDMa_1500_750_1.txt')

# Plotting Regions

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(13,10) )
MT_vs_sdmass_1500_750.plot(counts=True, equidistant='xy', counts_formatter="{:.2e} \n$\pm$\n {:.2e}".format, counts_fontsize=10,)
ax.set_xlabel(r'$M_{T}$')
ax.set_ylabel(r'$softdrop\ mass$')
#fig.savefig('/home/users/ewallace/public_html/HbbMET/plots/MT_vs_sdmass_1750_750.png')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(13,10) )
MT_vs_sdmass_tt.plot(counts=True, equidistant='xy', counts_formatter="{:.2e} \n$\pm$\n {:.2e}".format, counts_fontsize=10,)
ax.set_xlabel(r'$M_{T}$')
ax.set_ylabel(r'$softdrop\ mass$')
#fig.savefig('/home/users/ewallace/public_html/HbbMET/plots/MT_vs_sdmass_tt.png')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(13,10) )
MT_vs_sdmass_W.plot(counts=True, equidistant='xy', counts_formatter="{:.2e} \n$\pm$\n {:.2e}".format, counts_fontsize=10,)
ax.set_xlabel(r'$M_{T}$')
ax.set_ylabel(r'$softdrop\ mass$')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(13,10) )
MT_vs_sdmass_QCD.plot(counts=True, equidistant='xy', counts_formatter="{:.2e} \n$\pm$\n {:.2e}".format, counts_fontsize=10,)
ax.set_xlabel(r'$M_{T}$')
ax.set_ylabel(r'$softdrop\ mass$')